# Emission Measure Slopes
Compute emission measure slopes from average intensities

In [1]:
import os
import sys

import numpy as np
from scipy.interpolate import splev
import matplotlib
import matplotlib.pyplot as plt
from sunpy.map import Map
import astropy.units as u
from astropy.coordinates import SkyCoord
import distributed
import astropy.time
import sunpy.time

import synthesizAR
from synthesizAR.instruments import InstrumentSDOAIA

%matplotlib inline

## Compute Average Intensities
Need to do this on Nikola because this is where the data lives. We want to compute the average intensities for all channels for all three nanoflare heating functions plus the observations

In [2]:
sys.path.append(os.path.join(os.environ['HOME'], 'projects/synthesized_timelag_maps/scripts/'))
from aiacube import DistributedAIACube,DistributedAIACollection

In [3]:
cluster = distributed.LocalCluster(n_workers=64,threads_per_worker=1)
client = distributed.Client(cluster)
client

Client Scheduler: tcp://127.0.0.1:35802 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 64 Cores: 64 Memory: 270.38 GB


First the models

In [4]:
channels = [94,131,171,193,211,335]
read_template = '/storage-home/w/wtb2/data/timelag_synthesis_v2/{}_frequency/nei/SDO_AIA/{}/map_t{:06d}.fits'
save_template = '/storage-home/w/wtb2/data/shine2018/{}_{}_time_average.fits'

In [9]:
for f in ['high','intermediate','low']:
    for c in channels:
        cube = DistributedAIACube.from_files([read_template.format(f,c,i) for i in range(500,2500)])
        cube.average().save(save_template.format(f,c))

then the data

In [5]:
read_template_data = '/storage-home/w/wtb2/data/timelag_synthesis_v2/observational_data/aia/cutouts/aia_lev1.5_201102*_{}_cutout.fits'

In [6]:
for c in channels:
    cube = DistributedAIACube.from_files(read_template_data.format(c))
    cube.average().save(save_template.format('observed',c))

## Compute Emission Measures
Compute the DEM using the method of Hannah and Kontar (2012)